# Endpoint en Tiempo Real.

- Para probar los endpoits podemos usar boto3 o AWS CLI.

```bash
aws sagemaker-runtime invoke-endpoint \
    --endpoint-name endpoint_name \
    --body fileb://$file_name \
    output_file.txt
```
- También desde la interfaz de sagemaker studio.
- Doc: https://docs.aws.amazon.com/es_es/sagemaker/latest/dg/realtime-endpoints-test-endpoints.html

#### Datos de prueba

In [ ]:
import pandas as pd
df_test = pd.read_csv('test.csv', header=None)

In [ ]:
to_try = df_test.iloc[[1,2, 497, 499], :]
to_try

#### Llamada al endpooint en tiempo real

In [ ]:
csv_string = to_try.iloc[:, 1:].to_csv(index=False, header=False)
csv_string

In [ ]:
import boto3

# Create a low-level client representing Amazon SageMaker Runtime
sagemaker_runtime = boto3.client("sagemaker-runtime")

# The name of the endpoint. The name must be unique within an AWS Region in your AWS account. 
endpoint_name='chun-xgboost-severless'

# After you deploy a model into production using SageMaker hosting 
# services, your client applications use this API to get inferences 
# from the model hosted at the specified endpoint.
response = sagemaker_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='text/csv',
    Body=csv_string,
)

# Optional - Print the response body and decode it so it is human read-able.
print(response['Body'].read().decode('utf-8'))

### Invocación desde una función lambda
- https://aws.amazon.com/blogs/machine-learning/call-an-amazon-sagemaker-model-endpoint-using-amazon-api-gateway-and-aws-lambda/

In [ ]:
import os
import io
import boto3
import json
import csv

# grab environment variables
ENDPOINT_NAME = 'chun-xgboost-severless'
runtime= boto3.client('runtime.sagemaker')

def lambda_handler(event, context):
    print("Received event: " + json.dumps(event, indent=2))
    
    data = json.loads(json.dumps(event))
    payload = data['data']
    print(payload)
    
    response = runtime.invoke_endpoint(EndpointName=ENDPOINT_NAME,
                                       ContentType='text/csv',
                                       Body=payload)
    print(response)
    result = json.loads(response['Body'].read().decode())
    print(result)
    pred = int(result['predictions'][0]['score'])
    predicted_label = 'M' if pred == 1 else 'B'
    
    return predicted_label